In [1]:
stt_pecha_tools = "/home/monlamai/Documents/GitHub/combining-dataset/stt_pecha.tsv"
prodigy = "/home/monlamai/Documents/GitHub/combining-dataset/prodigy.tsv"
mv = "/home/monlamai/Documents/GitHub/saymore-report-generator/elan_to_segments/stt_mv.tsv"

In [2]:
old_mv_test = "/home/monlamai/Downloads/test.tsv"
old_mv_train = "/home/monlamai/Downloads/train.tsv"
old_mv_validation = "/home/monlamai/Downloads/validation.tsv"

In [3]:
import pandas as pd
old_mv_test_df = pd.read_csv(old_mv_test, sep='\t')
old_mv_train_df = pd.read_csv(old_mv_train, sep='\t')
old_mv_validation_df = pd.read_csv(old_mv_validation, sep='\t')

In [4]:
import pandas as pd
pecha_tools_df = pd.read_csv(stt_pecha_tools, sep='\t')
prodigy_df = pd.read_csv(prodigy, sep='\t')
mv_df = pd.read_csv(mv, sep='\t')

In [5]:
df = pd.concat([pecha_tools_df, prodigy_df, mv_df, old_mv_test_df, old_mv_train_df, old_mv_validation_df], ignore_index=True)

# clean the combined tsv

In [6]:
import re

def replace_repeated_sequences(text):
    # Match repeated word sequences with at least one word
    pattern = r'\b(\w+(?:\s+\w+)*)(?=\s+\1\b)'
    return re.sub(pattern, "", text)

# Example
text = "This is is a test test test. Hello hello world. Python python is awesome!"
print(replace_repeated_sequences(text))



This  is a   test. Hello hello world. Python python is awesome!


In [7]:

text = "ཧ་ཧ་ཧ་ཧ་ཧ་འེ་"
text = re.sub(r'ཧ་ཧ་(ཧ་)+', r'ཧ་ཧ་ཧ་', text)
print(text)

ཧ་ཧ་ཧ་འེ་


In [8]:
import re
def clean_transcription(text):
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = text.strip()

    text = re.sub(r"་+", "་", text)
    text = re.sub(r"།+", "།", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\s+།", "།", text)

    text = re.sub(r"ཧཧཧ+", "ཧཧཧ", text)
    text = re.sub(r'འེ་འེ་(འེ་)+', r'འེ་འེ་འེ་', text)
    text = re.sub(r'ཧ་ཧ་(ཧ་)+', r'ཧ་ཧ་ཧ་', text)

    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\/\{\}\(\)\༽\》\༼\《\༅\༄\༈\༑\༠]'
    
    text = re.sub(chars_to_ignore_regex, '', text)+" "
    return text

def check_if_regex(text):
    text = str(text)
    regex = re.compile(r'[a-zA-Z]+')
    match = re.search(regex, text)
    
    # Check if a match was found.
    if match:
        return True
    else:
        return False

def add_prefix(path):
    return '/home/monlamai/Documents/HuggingFace/tibetan_voice/audio/wav/' + path

In [9]:
df.shape

(639510, 5)

In [10]:
df = df[df['uni'].str.len() < 994]

In [11]:
df = df[~df['uni'].apply(check_if_regex)]

In [12]:
df.shape

(638278, 5)

In [14]:
df['uni'] = df['uni'].map(clean_transcription)

In [15]:
import pyewts
converter = pyewts.pyewts()

df['wylie'] = df['uni'].map(converter.toWylie)

In [16]:
df.head()


,file_name,uni,wylie,url,dept
0,STT_AB00236_0089_766379_to_777101,བྱས་ཤིག་ཞེས་བཤད་དེ་དོན་གྲུབ་འཁྲིས་ཀྱི་ཤུར་ལྕོག...,byas shig zhes bshad de don grub 'khris kyi sh...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
1,STT_TT00498_0182_1009491_to_1011795.wav,ང་རང་འཇོན་པོ་སང་ཙང་། རིག་པ་ཡག་པོ་སང་ཙང་།,nga rang 'jon po sang tsang /_rig pa yag po sa...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,STT_TT
2,STT_AB00236_0093_798284_to_804308,དོན་གྲུབ་ཀྱིས་ར་སྤུར་མཆིལ་མ་ཕྲན་ཙམ་འདོར་ཏེ་ལག་...,don grub kyis ra spur mchil ma phran tsam 'dor...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
3,STT_TT00501_0036_191445_to_195445.wav,གནས་རྩ་ཆེན་ཞེ་བོ་ཅིག་དཔེ་གཅིག ད་ལྷ་བླ་མའི་གདན་...,gnas rtsa chen zhe bo cig dpe gcig_da lha bla ...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,STT_TT
4,STT_AB00236_0094_804735_to_811203,དེ་རིང་ཁྱེད་ཚོའི་ཚོགས་ཆུང་ལ་ཡ་ལས་ཀ་མི་འདུག་གེ་...,de ring khyed tsho'i tshogs chung la ya las ka...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB


In [17]:
df['url'] = df['url'].map(lambda x : x.replace('#','%23'))

In [18]:
df = df[df['file_name'] != 'STT_AB00148_0687_2124469_to_2126579']

In [19]:
df.drop_duplicates(subset='file_name', keep="first", inplace=True)

In [20]:
df.to_csv('all.tsv', sep='\t', index=False)

In [21]:
df.head()

,file_name,uni,wylie,url,dept
0,STT_AB00236_0089_766379_to_777101,བྱས་ཤིག་ཞེས་བཤད་དེ་དོན་གྲུབ་འཁྲིས་ཀྱི་ཤུར་ལྕོག...,byas shig zhes bshad de don grub 'khris kyi sh...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
1,STT_TT00498_0182_1009491_to_1011795.wav,ང་རང་འཇོན་པོ་སང་ཙང་། རིག་པ་ཡག་པོ་སང་ཙང་།,nga rang 'jon po sang tsang /_rig pa yag po sa...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,STT_TT
2,STT_AB00236_0093_798284_to_804308,དོན་གྲུབ་ཀྱིས་ར་སྤུར་མཆིལ་མ་ཕྲན་ཙམ་འདོར་ཏེ་ལག་...,don grub kyis ra spur mchil ma phran tsam 'dor...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB
3,STT_TT00501_0036_191445_to_195445.wav,གནས་རྩ་ཆེན་ཞེ་བོ་ཅིག་དཔེ་གཅིག ད་ལྷ་བླ་མའི་གདན་...,gnas rtsa chen zhe bo cig dpe gcig_da lha bla ...,https://d38pmlk0v88drf.cloudfront.net/stt_pech...,STT_TT
4,STT_AB00236_0094_804735_to_811203,དེ་རིང་ཁྱེད་ཚོའི་ཚོགས་ཆུང་ལ་ཡ་ལས་ཀ་མི་འདུག་གེ་...,de ring khyed tsho'i tshogs chung la ya las ka...,https://huggingface.co/datasets/openpecha/STT_...,STT_AB


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Splitting the data
train, test_temp = train_test_split(df, test_size=0.1, random_state=42, stratify=df['dept'])
val, test = train_test_split(test_temp, test_size=0.5, random_state=42, stratify=test_temp['dept'])         

In [23]:
len(train), len(val), len(test), len(train)+len(val)+len(test), len(df)

(557511, 30973, 30973, 619457, 619457)

In [19]:
len(train), len(val), len(test), len(train)+len(val)+len(test), len(df)

(486175, 27010, 27010, 540195, 540195)

In [24]:
train.to_csv('550-train.tsv', sep='\t', index=False)
val.to_csv(  '550-val.tsv', sep='\t', index=False)
test.to_csv( '550-test.tsv', sep='\t', index=False)

In [25]:
df['uni'].str.contains('ཧཧཧ').value_counts()

uni
False    618892
True        565
Name: count, dtype: int64

In [1]:
import pandas as pd

df = pd.read_csv('all.tsv', sep='\t')

In [26]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0


In [27]:
df['time_span'] = df['file_name'].map(getTimeSpan)

In [28]:
df.groupby('dept')['time_span'].sum()/60/60

dept
STT_AB     75.318115
STT_CS     69.865409
STT_MV    283.327197
STT_NS     93.643877
STT_TT    136.281770
Name: time_span, dtype: float64

In [29]:
df['time_span'].sum()/60/60

658.4363680555556